# Sessie 9 - MCMC

Voer de onderstaande code uit.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import random
import scipy.stats as st

def plot_density(pdf, title=""):
    xs = np.linspace(0, 1, 1000)
    plt.plot(xs, list(map(pdf, xs)), 'b')
    plt.title(title);
    plt.xlabel("Theta");
    plt.show()

## Ziekte prevalentie

Gegeven een menselijke populatie, weten we dat er over een jaar een fractie $\theta$ van deze mensen griep zullen krijgen.
We willen nu op een correct Bayesiaanse wijze deze fractie berekenen, gegeven een aantal datapunten.
 
Wat we weten is dat 10 mensen jouw facebook poll hebben beantwoord. Deze poll vertelt ons dat maar 1 van deze personen griep heeft gehad vorig jaar.

In [ ]:
population = 100
infected = 10

Gegeven deze datapunten $D$, kunnen we nu de posterior distributie van $\theta$ berekenen.

$$P(\theta\ |\ D) \sim P(D\ |\ \theta) P(\theta)$$

Hiervoor hebben we een prior en likelihood functie nodig:
* We nemen een 'flat' prior, omdat deze vaak niet-informatief is en we a priori niets weten over de onderliggende distributie. Dit komt overeen met een beta-distributie met parameters $(\alpha = 1, \beta = 1)$, aangezien deze uniform is over het interval $[0, 1]$.
* We nemen de binomiaal distributie voor de likelihood. Dit lijkt een goede keuze, aangezien deze de kans op een bepaald aantal successen (in dit geval, mensen met griep) modelleert, gegeven een verwachte fractie $p$ van successen in $N$ experimenten.

In dit specifiek geval, als we exact deze prior en likelihood functie gebruiken, is de posterior analytisch op te lossen.
Meestal zal er dus daarentegen geen analytische oplossing bestaan.

Voor dit specifiek voorbeeld hebben we dus in principe MCMC niet nodig. 
We kunnen via deze weg wel de 'echte' posterior vergelijken met onze benadering.

In [ ]:
a, b = 1, 1
prior = lambda theta: st.beta.pdf(theta, a, b)
lik = lambda theta: st.binom.pmf(infected, population, theta) if 0 <= theta <= 1 else 0
post = lambda theta: st.beta.pdf(theta, a + infected, b + population - infected)

plot_density(prior, "Prior");
plot_density(lik, "Likelihood");
plot_density(post, "Posterior (deze heb je in principe niet)");

Bepaal nu het aantal iteraties, de 'target' distributie en 'proposal' distributie nodig om de posterior te benaderen met de MCMC methode. Aangezien de initialisatie van het algoritme een bias induceert op de eerste aantal samples, kunnen we ook een aantal burn-in samples opgeven die niet in rekening gebracht zullen worden om de uiteindelijke posterior te bepalen.

In [ ]:
target = lambda theta: lik(theta)*prior(theta)
proposal = lambda theta: theta + np.random.normal(0, 0.1)
iters = 5000
burn_in = 100

def metropolis(target, proposal, iters):
    theta = 0.5 # initiele guess op de x-as
    samples = [theta] # lijst van samples
    for i in range(iters): # loop voor een aantal iteraties
        # stap 1: stel een nieuwe waarde op basis van de huidige waarde
        theta_p = proposal(theta) 
        # stap 2: bereken de acceptatie ratio
        rho = min(1, target(theta_p) / target(theta))
        # stap 3: accepteer of verwerp de nieuwe waarde
        u = np.random.uniform()
        if u < rho:
            theta = theta_p
        # stap 4: sla de waarde op
        samples.append(theta)
    return samples

samples = metropolis(target, proposal, iters)
plt.hist(samples[burn_in:], 50, density=1, facecolor='green', alpha=0.6);
plot_density(post)

---

## Oefening 2: Niet-analytische integraal

Als we essentiele informatie hebben over de onderliggende distributie, is het handig om deze in de prior te encoderen. Dit maakt de prior informatief, maar kan de normaliserende factor in Bayes' rule niet analytisch integreerbaar maken.
Deze gevallen maakt de MCMC methode een populaire techniek.

Neem bijvoorbeeld de volgende prior:

In [ ]:
prior = lambda theta: 2 - abs(theta * 4 - 2) if 0 <= theta <= 1 else 0
plot_density(prior, "Prior");

Pas de MCMC methode toe op deze prior.

In [ ]:
target = lambda theta: lik(theta)*prior(theta)
samples = metropolis(target, proposal, iters)
plt.hist(samples[burn_in:], 50, density=1, facecolor='green', alpha=0.6);
plot_density(post)

<b>VRAAG:</b>

Je ziet dat er discrepantie tussen de posterior distributies met en zonder informatieve prior. 

- Waarom?
- Hoe zou je het effect van de priors kunnen wegkrijgen? 
- Toon dit aan door de bovenstaande code aan te passen.

<b>ANTWOORD:</b>

- Probeer de variabelen 'population' en 'infected' eens respectievelijk op 100 en 10 te zetten.
- Dit komt overeen met het aantal samples te verhogen, waardoor de geobserveerde data (likelihood) meer impact heeft dan vooraf aangenomen informatie (prior).

---

## Oefening 3: Metropolis-Hastings algoritme

In wat volgt zullen we het Metropolis-Hastings algoritme implementeren aan de hand van een case. Het is de bedoeling dat aan het einde van deze oefening je een begrip hebt van hoe het algoritme in elkaar zit en wat de verschillende onderdelen zijn.

#### Introductie

De case is de volgende:

Koning Markov is koning van een eilandengroep bestaande uit 5 eilanden. Eén van zijn verplichtingen als koning is dat hij op elk eiland een bepaalde tijd verblijft, en dat die tijd proportioneel is aan de populatie van het eiland. 

De koning wil niet bijhouden hoe vaak en hoe lang hij elk van de eilanden al bezocht heeft. Bovendien heeft hij een hekel aan routine, m.a.w. hij wil niet elke nacht weten op welk eiland hij de volgende nacht zal zijn. 

Zijn adviseur Metropolis moet een systeem vinden dat voldoet aan de eisen van het koningschap, en aan de volgende regels:
* Het systeem kiest de eilanden waar de koning 's nachts verblijft ad random.
* Het systeem vereist niet dat de koning zich herinnert waar hij in het verleden verbleven heeft.
* Het systeem vereist niet dat de koning de omvang van de populatie van de eilanden onthoudt.
De koning kan wel aan de klerk vragen wat de omvang van de populatie van het eiland is waarop hij verblijft. De populatieinfo die verkregen kan worden van de andere eilanden is beperkt. Het neemt immers een halve dag in beslag om te zeilen van één eiland naar een andere. Bijgevolg is de hoeveelheid informatie die hij kan verzamelen in één dag dus beperkt.

Metropolis bedacht het volgende systeem (in de veronderstelling dat de koning regeert over 5 eilanden):
* Iedere morgen vraagt de koning aan de klerk wat de populatie is van het eiland waar ze momenteel verblijven.
* Vervolgens wordt er ad random één van de vier andere eilanden gekozen en reizen ze er naar toe tijdens de voormiddag.
    * Stel J(b|a) is de conditionele probabiliteit om eiland b te selecteren als eiland a het eiland is waarop ze momenteel verblijven.
    * J is niet afhankelijk van de omvang van de populaties op de eilanden aangezien de koning die niet kan herinneren.
* Tijdens de lunch op het voorgestelde eiland (dit is het eiland naar waar ze tijdens de voormiddag gevaren waren), vragen ze naar de bevolkingsomvang van het eiland.
    * Als de bevolking van het voorgestelde eiland groter is dan die van het eiland waarvan ze in de morgen vertrokken waren, dan blijven ze op het eiland voor de nacht.
    * Als de bevolking kleiner is dan blijven ze met een probabiliteit A op het voorgestelde eiland, anders keren ze terug naar het vorige eiland.
    
Metropolis is er van overtuigd dat voor de correcte keuzes van J en A, de regels van het koningschap voldaan zullen zijn.

<b>VRAAG:</b>

Metropolis stelt echter wel vast dat A niet 0 of 1 mag zijn. Waarom?*

<b>ANTWOORD:</b>

Indien A = 0 dan zal de koning altijd terugkeren naar het vorige eiland, ongeacht de verhouding van de bevolkingsomvangen van het voorgestelde en het vorige eiland. Het omgekeerde geldt als A = 1.

#### Implementatie

In [ ]:
def jump_to_any_island(current: int, other: int, count: int) -> float:
    
    """
    Deze functie geeft aan iedere huidige eiland -> voorgestelde eiland sprong
    een gelijke probabiliteit.
    
    Parameters
    ----------
    current: int
        current is de index van het huidige eiland.
    other: int
        other is de index van het andere eiland.
    count: int
        count is het aantal eilanden waarover de koning regeert.
    
    Raises
    ------
    None
    
    Returns
    -------
     De probabiliteit dat de koning van het huidige eiland (current) naar het andere eiland (other) springt
        PS: we willen hier een uniforme verdeling, dus de kans dat de koning van eiland 1 naar eiland 2 springt is gelijk aan de kans dat hij van eiland 2 naar eiland 1 springt.
    """
    
    return 1 / (count - 1)

In [ ]:
def koning_markov(num_steps: int, population: list, island_names: list, start: [int, str], J: "pyObj") -> pd.DataFrame:
    
    """
    Deze functie bepaalt de eilanden die de koning bezoekt.
    
    Parameters
    ----------
    num_steps: int
        Aantal iteraties/nachten van de koning.
    population: list
        Een list dat de bevolkingsomvang voor elk eiland bevat.
    island_names: list
        Een list dat de namen van de eilanden bevat. De namen kunnen
        integers of strings zijn.
    start: int/str
        Bepaalt het eiland van waar de simulatie start. De waarde 
        moet overeenkomen met een naam van een eiland.
    J:
        J is een functie die bepaalt wat de probabiliteit is dat er
        gesprongen wordt naar een bepaalt eiland.
    
    Returns
    -------
    list
        een lijst met de bezochte eilanden bevat
    """
    
    num_islands = len(population) # aantal eilanden
    islands_seq = [start] # de volgorde van bezochte eilanden, i.e. 'de samples'
    current = island_names.index(start) # initiele guess => het eiland waar de koning zich bevindt
    
    # For loop die één voor één de nachten simuleert.
    for _ in range(num_steps):
        
        # Waar kan de koning naartoe springen?
        other_islands = list(set(list(range(num_islands))) - set([current])) 
        
        # stap 1: ad random een nieuw eiland kiezen op basis van de populatie hoeveelheiden
        weights = [J(current, other_islands[i], len(population)) for i in range(len(other_islands))]
        next = random.choices(other_islands, weights=weights)[0]
       
        # stap 2: berekenen de acceptatie ratio
        # met andere woorden: de probabiliteit om op een voorgestelde eiland te blijven.
        # -> hoe groter de populatie van het nieuw eiland -> hoe kleiner de kans dat de koning daar blijft.
        prob_move = population[next] / population[current]

        # stap 3: accepteer of verwerp de nieuwe waarde
        # met andere woorden: de koning blijft op het huidige eiland of hij springt naar het voorgestelde eiland.
        u = np.random.uniform()
        if u < prob_move:
            current = next

        # bijhouden van huidig eiland, i.e. de 'sample' bijhouden
        islands_seq.append(current)
    
    volgorde = [island_names[island_idx] for island_idx in islands_seq[1:]]
    return volgorde

***Vraag:***

*Vraag: Welke distributie (de frequentie dat elk eiland bezocht werd) verwachten we voor de situatie waarin er 5 eilanden zijn (1, 2, 3, 4, 5) met de volgende bevolking (1, 2, 3, 4, 5)? Denk hierbij aan de regels die opgelegd zijn aan het koningschap.*

- Antwoord: Het koningschap vereist dat de tijd dat de koning spendeert op een eiland proportioneel is aan de bevolkingsomvang van het eiland. 

- Dit betekent het volgende:
    * Eiland 1: 1 / (1+2+3+4+5) = 1/15 = 0.067
    * Eiland 2: 2 / (1+2+3+4+5) = 2/15 = 0.133
    * Eiland 3: 3 / (1+2+3+4+5) = 3/15 = 0.2
    * Eiland 4: 4 / (1+2+3+4+5) = 4/15 = 0.267
    * Eiland 5: 5 / (1+2+3+4+5) = 5/15 = 0.333

- Metropolis-Hastings convergeert naar deze verdeling, indien de voorwaarden voor een stationaire distributie van een Markov Chain voldaan zijn (e.g. de stationaire distributie bestaat en is uniek (dit laatste is gegarandeerd door de ergodiciteit van het Markov process).

*Vraag: Wat stellen we vast ivm de verdeling van de eilanden die bezocht werden?*

- Antwoord: We zien dat het aantal nachten dat de koning de eilanden bezoekt de verdeling goed benadert. De benadering is uiteraard niet perfect. Deze wordt beter naarmate meer iteraties/nachten gesimuleerd worden.

In [ ]:
# Runs een simulatie waarbij er naar eender welk eiland gesprongen mag worden.
eiland_volgorde = koning_markov(5000, [1,2,3,4,5], [1,2,3,4,5], 2, jump_to_any_island)
df = pd.DataFrame(list(eiland_volgorde))
df.columns = ["data"]


In [ ]:
# Visualiseert de frequentie dat de eilanden bezocht werden.
fig = px.histogram(df, 
                   x="data", 
                   title="De frequentie van hoe vaak een eiland bezocht werd", 
                   histnorm='probability density')
fig.show()

### ALTERNATIEF: Wat als de koning enkel wijzerszin mag gaan?

*Vraag: Wat stellen we vast ivm de verdeling van de eilanden die bezocht werden?*

- Antwoord: We zien dat het algoritme helemaal niet de correcte verdeling benadert. Dit heeft te maken met het feit dat de voorwaarden van irreduceerbaarheid en aperiodiciteit geschonden zijn. De proposal distributie die we in de eerste situatie gebruikt hebben is symmetrisch. Alle eilanden hebben een zelfde kans om gekozen te worden. In de tweede situatie gebruiken we een asymmetrische proposal distributie. Het Metropolis-Hastings algoritme kan hiermee overweg. Maar, de proposal distributie schendt echter een aantal belangrijke voorwaarden ivm de Markov Chain. Dit leidt tot problemen met de convergentie.

In [ ]:
def jump_only_clockwise(current: int, other: int, count: int) -> float:
    
    """
    Deze functie laat enkel sprongen in wijzerzin toe (wijzerzin wordt hier 
    geinterpreteerd als index 0 -> 1 -> 2 -> 3 -> ... -> 0 -> ...).
    
    Parameters
    ----------
    current: int
        current is de index van het huidige eiland.
    other: int
        other is de index van het andere eiland.
    count: int
        count is het aantal eilanden waarover de koning regeert.

    Returns
    -------
    float:
        De probabiliteit dat de koning van het huidige eiland (current) naar het andere eiland (other) springt
    """
    
    # De index van het huidige (current) island wordt verhoogd met 1.
    # Via de modulo operator bekomen we dan de index van het volgende
    # eiland (in wijzerzin). 
    index_of_next_island = (current + 1) % count
    
    return 1.0 if other == index_of_next_island else 0.0

In [ ]:
# Runs een simulatie waarbij er enkel in wijzerzin gesprongen mag worden.
clockwise_result = koning_markov(5000, [1,2,3,4,5], [1,2,3,4,5], 2, jump_only_clockwise)
df2 = pd.DataFrame(list(clockwise_result))
df2.columns = ["data"]


In [ ]:
# Visualiseert de frequentie dat de eilanden bezocht werden.
fig = px.histogram(df2, 
                   x="data", 
                   title="De frequentie van hoe vaak een eiland bezocht werd (clockwise)", 
                   histnorm='probability density')
fig.show()